In [ ]:
import config
import gzip
import lib_db
import lib_nlp
import datetime
import lib_vk
import datetime
import lib_process
import lib.reliability_rating as reliability_rating
from random import shuffle
import pandas as pd

In [ ]:
urls = {'bank':  {'vk': 'bank', 'fb': 'bank', 'twitter': 'bank'}}

In [ ]:
import vk_api
access_token = ''
vk_session = vk_api.VkApi(token = access_token)
vk = vk_session.get_api()
lib_vk.vk = vk

In [ ]:
def get_reliability(gid, count=50):
    users = lib_vk.get_group_users(gid, iterations=50, debuglines=True, skip_cache=False, good_sampling_offset=True)
    reliability_bank = []
    shuffle(users)
    good_count = 0
    active_count = 0
    for u in users[:count]:
        r, p_count = reliability_rating.reliability(id=u, need_post_count=True)
        reliability_bank.append(r)
        if r > 0.2:
            good_count += 1
        if p_count > 0:
            active_count += 1
    return reliability_bank, good_count/count, active_count/count

In [ ]:
name = 'bank'

In [ ]:
%%time
res = {}
res_active = {}
for name in urls:
    rel_data, rel_rating, active_rating = get_reliability(urls[name]['vk'], count=200)
    res[name] = rel_rating
    res_active[name] = active_rating
    print(name, rel_rating, active_rating)

In [ ]:
import pandas

In [ ]:
df = pd.DataFrame.from_dict(res, orient='index')

In [ ]:
activity_column = []
for i in df.index:
    print(i)
    activity_column.append(res_active[i])

In [ ]:
df['active'] = activity_column

In [ ]:
df.sort_values('active', ascending=False)

In [ ]:
import matplotlib as mpl
# mpl.rc('font', family='Verdana') 
mpl.rc('font', family='DejaVu Sans') 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
cnt = 0
def draw_scatter(res, res_active):
    points_x = []
    points_y = []
    names = []
    sizes = []
    sizes2 = []
    for name in res.keys():
        points_x.append(res_active[name])
        points_y.append(res[name])
        names.append(name)
        sizes.append(500)
#         sizes2.append((d['post_count']) * 2)
#     plt.subplots_adjust(bottom = 0.1)
#     ax.scatter(points_x, points_y, marker='o', c=range(len(sizes)), s=sizes, cmap=plt.get_cmap('Spectral'))
#     ax.scatter(points_x, points_y, marker='o', c=[20]*(len(sizes)), s=sizes, cmap=plt.get_cmap('Greys'))
    ax.scatter(points_x, points_y, marker='o', c=[0]*(len(sizes)), s=sizes, cmap=plt.get_cmap('Spectral'))
    ax.set_title('Анализ аудитории')
    ax.set_xlabel("активность")
    ax.set_ylabel("достоверность")

    for label, x, y in zip(names, points_x, points_y):
        ax.annotate(
            label,
            xy=(x, y), xytext=(-20, 20),
            textcoords='offset points', ha='right', va='bottom',
            bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
            arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))

#     plt.subplots_adjust(left=3, right=4, top=3.5, bottom=2)

In [ ]:
plt.figure()
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(1,1,1) # one row, one column, first plot
draw_scatter(res, res_active)